Cluster the orange ball using Single Guassian

In [1]:
# import packages
import cv2
import os
import numpy as np
import math

In [2]:
def cal_mean_cov(img):
    l, w, h = img.shape
    mean = [[np.sum(img[:,:,0])/(l*w)],[np.sum(img[:,:,1])/(l*w)],[np.sum(img[:,:,2])/(l*w)]]
    cov = np.zeros((3,3),)
    #R_value = []
    for width in range(len(img[:,0,0])):
        for length in range(len(img[0,:,0])):
            RGB_value = [[img[width][length][0]],[img[width][length][1]],[img[width][length][2]]]
            cov = cov + (np.asmatrix(RGB_value) - np.asmatrix(mean))@(np.asmatrix(RGB_value) - np.asmatrix(mean)).T
            #R_value.append(img[width][length][0])
    cov = cov/(l*w)
    #print("size of image: ", l*w)
    #print("variance for R is: ", np.var(R_value))
    return mean,cov

In [ ]:
# load data
dir = "train_images" # path to the train image dataset
# User defined threshold
tau = 0.00002

for img_name in os.listdir(dir):
    img = cv2.imread(os.path.join(dir, img_name))
    mean, cov = cal_mean_cov(img)
    # creat a mask to indicate the position of ball
    mask = np.zeros((img.shape[:-1]))
    print("cov: \n", cov)
    prior = 0.5
    likelihood_list = []
    for width in range(len(img[:,0,0])):
        for length in range(len(img[0,:,0])):
            curr_pixel = np.asmatrix([[img[width][length][0]],[img[width][length][1]],[img[width][length][2]]])
            likelihood = 1/(math.sqrt(((2*math.pi)**3)* np.linalg.det(cov))) * math.exp((-0.5)*(curr_pixel-mean).T @ np.linalg.inv(cov)@ (curr_pixel-mean))
            #print("likelihood", likelihood)
            likelihood_list.append(likelihood)
            if (likelihood * prior > tau):
                mask[width][length] = 1
    ##  show mask
    three_d_mask = np.asarray([mask,mask,mask])
    three_d_mask = np.stack((mask,mask,mask),axis=2)
    print("size of mask ", three_d_mask.shape)
    cv2.imshow("image", img)
    cv2.waitKey(0)
    print("max of likelihood ", max(likelihood_list))
    break


cov: 
 [[2781.61681836 2878.44650827 2934.94459974]
 [2878.44650827 3007.86237752 3083.16294443]
 [2934.94459974 3083.16294443 3227.16865765]]
size of mask  (640, 480, 3)


In [ ]:
mask = np.zeros((480,960))
print(mask.shape)
three_d_mask = np.stack((mask,mask,mask),axis=2)
three_d_mask.shape